In [1]:
import pandas as pd
import pyomo.environ as pyo
import networkx as nx
import matplotlib.pyplot as plt
import random
from random import sample

In [2]:
nodes = pd.read_pickle("../../data/mock/mock_nodes.pkl")
channels = pd.read_pickle("../../data/mock/mock_channels.pkl")

In [3]:
nodes.to_csv("../../data/mock/mock_nodes.csv")
channels.to_csv("../../data/mock/mock_channels.csv")

In [4]:
## Note that the following are arbitrary policies, that can be changed as needed.
## Deal with multi-edges (aka multiple channels between two peers):
## - average rate fee
## - average base fee,
## - average capacity
## - keep one of the two channel ids
aggregation_dict = {
    "channel_id": "first",
    "rate_fee": "mean",
    "base_fee": "mean",
    "capacity": "sum"
}
channels.reset_index(inplace=True)
channels = channels.groupby(["node1_pub", "node2_pub"]).agg(aggregation_dict)
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

In [5]:
## To delete from [outgoing] and [incoming] lists the multi-edge
for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "outgoing_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "outgoing_channels"] = chan

for i in nodes.index:
    chan = []
    for l in nodes.loc[i, "incoming_channels"]:
        if l in channels.index:
            chan.append(l)
    nodes.at[i, "incoming_channels"] = chan


**The execution of this chunck highlights how the channel directions is often wrong, even though the peers seem to be correct. There's probably an issue in the detection of the directed relationship.

- [ ] solve this issue

In [6]:
# Add to the nodes dataset the list of peers (nodes who have a channel with the N node)
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
for n in nodes.index:
    peers = []
    for c in nodes.loc[n, "outgoing_channels"]:
        source, destination = channels[channels["channel_id"]==c].index[0]
        print(f"For node {n}, the channel {c} is {source} --> {destination}")
        #try:
        #    assert source == n
        #    peers.append(destination)
        #except AssertionError as e:
        #    print(f"{source} is NOT equal to {n} for channel {c}")
    print(peers)
channels.reset_index(inplace=True) 
channels.set_index("channel_id", inplace=True)

For node FNRZTCUAZH, the channel 009566356891778588 is FNRZTCUAZH --> EZOBHZBFPU
For node FNRZTCUAZH, the channel 002022819420050336 is FNRZTCUAZH --> IUTMEIARAL
For node FNRZTCUAZH, the channel 111742423300785317 is FNRZTCUAZH --> RMCLDHZQZI
For node FNRZTCUAZH, the channel 308829925196470866 is FNRZTCUAZH --> HAVOHDULRN
For node FNRZTCUAZH, the channel 557332762177297363 is FNRZTCUAZH --> LHNDZFRFEI
[]
For node FQOCLOMAZN, the channel 448486103730170488 is FQOCLOMAZN --> VTRQTZMDZB
For node FQOCLOMAZN, the channel 910051561595375963 is FQOCLOMAZN --> PGHEECSUVU
For node FQOCLOMAZN, the channel 579406143782824321 is FQOCLOMAZN --> OQAPDLNBGO
For node FQOCLOMAZN, the channel 999703618160710949 is FQOCLOMAZN --> UNUBIMZHIL
For node FQOCLOMAZN, the channel INV075763090018637224 is FQOCLOMAZN --> VQPGEVSSQH
For node FQOCLOMAZN, the channel INV823213265093346966 is FQOCLOMAZN --> IDZCVBUINF
For node FQOCLOMAZN, the channel INV088754794959294646 is FQOCLOMAZN --> UENFNHHIMC
For node FQOCLOM

In [7]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [8]:

def create_demand(pd_object: pd.DataFrame) -> pd.DataFrame:
    """
    This function assigns the role of sender and receiver to
    two random nodes in the network
    :param pd_object: nodes dataframe
    :return: nodes dataset with demand column
    """
    #random.seed(874631)
    counterparties = sample(pd_object.index.to_list(), 2)
    sender = counterparties[0]
    receiver = counterparties[1]
    # Amounts in millisat (aka 10'000'000 is 10'000 sats)
    amount = 10000 #random.randint(a=10000000, b=30000000)

    print(
        f"Transaction of {amount} sats from {pd_object[pd_object.index == sender]['alias'].item()} to {pd_object[pd_object.index == receiver]['alias'].item()}.")

    pd_object["demand"] = 0
    pd_object.loc[pd_object.index == sender, "demand"] = -amount
    pd_object.loc[pd_object.index == receiver, "demand"] = amount

    return pd_object


nodes = create_demand(nodes)



Transaction of 10000 sats from UBNELAQEHH.com to VFFZRHMNPD.com.


In [9]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [10]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [11]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

In [12]:
## Single path constrain
#def single_path_rule(model: pyo.ConcreteModel, n):
#    outgoing_edges = nodes.loc[n, "outgoing_channels"]
#    incoming_edges = nodes.loc[n, "incoming_channels"]
#    
#    return sum(model.x[n, j].value for j in outgoing_edges) == 1 and sum(model.x[i, j].value for i, j in incoming_edges) == 1
#
##model.SinglePathConstraint= pyo.Constraint(model.NODES, rule=single_path_rule)

In [13]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * (1 if channels.loc[(a,b), "base_fee"]<1 else model.x[(a, b)])

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

In [14]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)        

In [15]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())



Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmp9o7yynj8.pyomo.lp -stat=1 -solve -solu /tmp/tmp9o7yynj8.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 98 (-690) rows, 688 (-691) columns and 1351 (-694) elements
Statistics for presolved model
Original problem has 689 integers (689 of which binary)
Presolved problem has 1 integers (1 of which binary)
==== 75 zero objective 16 different
==== absolute objective values 16 different
==== for integers 0 zero objective 1 different
1 variables have objective of 1
==== for integers absolute objective values 1 different
1 variables have objective of 1
===== end objective counts


Problem has 98 rows, 688 columns (613 with objective) and 1351 elements
There are 26 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 687 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of typ

In [16]:
ris = []
for i,v in model.a.extract_values().items():
    if v != 0:
        ris.append((i, v))

print(ris)


[(('BTPMLOZRGV', 'MSOMRDUAUM'), 10000.0), (('DOMHCDHEHC', 'TZBIIOFEHU'), 10000.0), (('MSOMRDUAUM', 'VFFZRHMNPD'), 10000.0), (('TZBIIOFEHU', 'ZMMPQABQVV'), 10000.0), (('UBNELAQEHH', 'DOMHCDHEHC'), 10000.0), (('ZMMPQABQVV', 'BTPMLOZRGV'), 10000.0)]


In [17]:
from decimal import Decimal
DF = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    DF.loc[c, "amount"] = Decimal(value)
    DF.loc[c, "source"] = index[0]
    DF.loc[c, "destination"] = index[1]
    c += 1

In [18]:
DF[DF["amount"]>0]

,amount,source,destination
17,10000,BTPMLOZRGV,MSOMRDUAUM
65,10000,DOMHCDHEHC,TZBIIOFEHU
282,10000,MSOMRDUAUM,VFFZRHMNPD
529,10000,TZBIIOFEHU,ZMMPQABQVV
537,10000,UBNELAQEHH,DOMHCDHEHC
661,10000,ZMMPQABQVV,BTPMLOZRGV


In [19]:
nodes.reset_index(inplace=True)
DF_nodes = nodes[nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "source"].tolist()) | nodes["pub_key"].isin(DF.loc[DF["amount"] > 0, "destination"].tolist())]
nodes.set_index("pub_key", inplace=True)

DF_nodes

,pub_key,alias,addresses,outgoing_channels,incoming_channels,demand
2,VFFZRHMNPD,VFFZRHMNPD.com,1,"[406095330371227284, 204235347547342220, 87278...","[347035514238996458, 680163780572874449, 94511...",10000
7,TZBIIOFEHU,TZBIIOFEHU.com,1,"[644952002657532143, 226105421905442682, 79118...","[460139081567358531, 913667246805953295, 14121...",0
15,UBNELAQEHH,UBNELAQEHH.com,1,"[776711399907502220, 980780786209232318, 09413...","[345375076210199852, 328735937661619616, INV77...",-10000
33,BTPMLOZRGV,BTPMLOZRGV.com,1,"[400503878448558488, 095912617911183726, 94158...","[400505013037845615, 337888811219778021, INV40...",0
34,MSOMRDUAUM,MSOMRDUAUM.com,1,"[385591007968304234, 835391766950894294, INV62...","[628480340283019775, 080177412121581207, 13512...",0
62,DOMHCDHEHC,DOMHCDHEHC.com,1,"[018889974500299251, 332118828068677068, 97562...","[354198096778059485, 914401952267255860, 73722...",0
74,ZMMPQABQVV,ZMMPQABQVV.com,1,"[707372786897802988, 378708731985246969, 92076...","[644952002657532143, 095912617911183726, 14477...",0
